# Effective Kraus Rank

In [2]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from loss_functions import *
from optimization import *
from quantum_maps import *
from quantum_tools import *
from experiments import *

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

C:\Users\krisw\Documents\ModelNISQ\notebooks\experiments\../../src_tf\experiments.py:264: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if init is "random":
C:\Users\krisw\Documents\ModelNISQ\notebooks\experiments\../../src_tf\experiments.py:268: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif init is "ideal":
C:\Users\krisw\Documents\ModelNISQ\notebooks\experiments\../../src_tf\experiments.py:278: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if povm is "random":
C:\Users\krisw\Documents\ModelNISQ\notebooks\experiments\../../src_tf\experiments.py:286: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif povm is "ideal":


## Real Hardware

In [33]:
n = 3
d = 2**n

In [34]:
inputs_map, inputs_spam, counts_map_list =  pickle.load(open("../../data/threeQubits_variational_3.p", "rb"))

targets_map = counts_to_probs(counts_map_list[:1000-6**n])
targets_spam = counts_to_probs(counts_map_list[1000-6**n:])


np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

spam_model = SPAM(d=d,
                  use_corr_mat=True,
                  optimizer = tf.optimizers.Adam(learning_rate=0.01))

spam_model.pretrain(targets = [init_ideal(d), povm_ideal(d)],
                num_iter = 300,
                verbose = False,
                )

spam_model.train(inputs = inputs_spam,
                 targets = targets_spam,
                 num_iter = 1000,
                 verbose = False,
                 )


kraus_model = KrausMap(d = d, 
                       rank = 8,
                       spam = spam_model,
                       )

model_L1 = ModelQuantumMap(
                        q_map = kraus_model,
                        loss = probs_loss,
                        optimizer = tf.optimizers.Adam(learning_rate=0.01),
                       )


model_L1.train(inputs = inputs_map,
               targets = targets_map,
               num_iter = 2000,
               verbose = False,
               )

  0%|          | 0/300 [00:00<?, ?it/s]

3.806583148768781e-05


  0%|          | 0/1000 [00:00<?, ?it/s]

0.00040995797244320096


  0%|          | 0/2000 [00:00<?, ?it/s]

0.0027734484936107195 0


In [35]:
inputs_map, inputs_spam, counts_map_list =  pickle.load(open("../../data/threeQubits_variational_3_2.p", "rb"))

targets_map = counts_to_probs(counts_map_list[:1000-6**n])
targets_spam = counts_to_probs(counts_map_list[1000-6**n:])


np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

spam_model = SPAM(d=d,
                  use_corr_mat=True,
                  optimizer = tf.optimizers.Adam(learning_rate=0.01))

spam_model.pretrain(targets = [init_ideal(d), povm_ideal(d)],
                num_iter = 300,
                verbose = False,
                )

spam_model.train(inputs = inputs_spam,
                 targets = targets_spam,
                 num_iter = 1000,
                 verbose = False,
                 )


kraus_model = KrausMap(d = d, 
                       rank = 8,
                       spam = spam_model,
                       )

model_L2 = ModelQuantumMap(
                        q_map = kraus_model,
                        loss = probs_loss,
                        optimizer = tf.optimizers.Adam(learning_rate=0.01),
                       )


model_L2.train(inputs = inputs_map,
               targets = targets_map,
               num_iter = 2000,
               verbose = False,
               )

  0%|          | 0/300 [00:00<?, ?it/s]

3.806583148768781e-05


  0%|          | 0/1000 [00:00<?, ?it/s]

0.00040238130576414355


  0%|          | 0/2000 [00:00<?, ?it/s]

0.002062635325599025 0


In [36]:
inputs_map, inputs_spam, counts_map_list =  pickle.load(open("../../data/threeQubits_variational_6.p", "rb"))

targets_map = counts_to_probs(counts_map_list[:1000-6**n])
targets_spam = counts_to_probs(counts_map_list[1000-6**n:])


np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

spam_model = SPAM(d=d,
                  use_corr_mat=True,
                  optimizer = tf.optimizers.Adam(learning_rate=0.01))

spam_model.pretrain(targets = [init_ideal(d), povm_ideal(d)],
                num_iter = 300,
                verbose = False,
                )

spam_model.train(inputs = inputs_spam,
                 targets = targets_spam,
                 num_iter = 1000,
                 verbose = False,
                 )


kraus_model = KrausMap(d = d, 
                       rank = 16,
                       spam = spam_model,
                       )

model_L12 = ModelQuantumMap(
                        q_map = kraus_model,
                        loss = probs_loss,
                        optimizer = tf.optimizers.Adam(learning_rate=0.01),
                       )


model_L12.train(inputs = inputs_map,
                targets = targets_map,
                num_iter = 2000,
                verbose = False,
                )

  0%|          | 0/300 [00:00<?, ?it/s]

3.806583148768781e-05


  0%|          | 0/1000 [00:00<?, ?it/s]

0.0004194076455761547


  0%|          | 0/2000 [00:00<?, ?it/s]

0.002526849762442614 0


In [37]:
choi1 = maps_to_choi([model_L1.q_map, model_L2.q_map])
choi2 = maps_to_choi([model_L12.q_map])

In [38]:
fid = (state_fidelity(choi1, choi2)/d**2).numpy()
print(fid)

0.8138528496453707
